In [7]:
import foolbox as fb
import art
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
import numpy as np
import joblib
import matplotlib.pyplot as plt 
import eagerpy as ep

print(tf.__version__)


2.6.3


In [8]:
X_train = joblib.load('../../preprocessed_data/InSDN/X_train.joblib')
X_test = joblib.load('../../preprocessed_data/InSDN/X_test.joblib')
Y_train = joblib.load('../../preprocessed_data/InSDN/Y_train.joblib')
Y_test = joblib.load('../../preprocessed_data/InSDN/Y_test.joblib')
print(Y_train)

[1. 0. 1. ... 1. 1. 1.]


In [9]:
from keras.models import load_model

cnn_loaded = load_model('../../models/cnn_insdn_model.keras')
lstm_loaded = load_model('../../models/lstm_insdn_model.keras')
ae_lstm_loaded = load_model('../../models/ae-lstm_insdn_model.keras')



### Defining evaluation metrics functions

In [10]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import time

def evaluate(true_labels, pred_labels, title):
    acc = accuracy_score(true_labels, pred_labels)
    prec = precision_score(true_labels, pred_labels, zero_division=0)
    rec = recall_score(true_labels, pred_labels, zero_division=0)
    f1 = f1_score(true_labels, pred_labels, zero_division=0)
    print(f"--- {title} ---")
    print(f"Accuracy : {acc:.4f}")
    print(f"Precision: {prec:.4f}")
    print(f"Recall   : {rec:.4f}")
    print(f"F1-score : {f1:.4f}")
    print()

### FGSM

In [ ]:
# from art.attacks.evasion import FastGradientMethod
# from art.estimators.classification import KerasClassifier

# # attack_fgsm = fgsmAttack(estimator=classifier)

# classifier = KerasClassifier(model=cnn_loaded)

# attack_fgsm = FastGradientMethod(estimator=classifier)

# adv_fgsm = attack_fgsm.generate(x=X_test[:2000], y=Y_test[:2000])

# # fgsm predictions
# preds_adv_fgsm = classifier.predict(adv_fgsm)
# labels_adv_fgsm = (preds_adv_fgsm > 0.5).astype(int).flatten()

# print("=== Prediction Counts ===")
# print("Original        :", Counter(labels_orig))
# print("fgsmAttack  :", Counter(labels_adv_fgsm))

from collections import Counter
from art.attacks.evasion import FastGradientMethod, ProjectedGradientDescent
from art.estimators.classification import KerasClassifier

# attack_fgsm = fgsmAttack(estimator=classifier)
classifier = KerasClassifier(model=cnn_loaded, clip_values=(X_test.min(), X_test.max()))
classifier._model.trainable = False

preds_clean = classifier.predict(X_test[:2000])
labels_clean = (preds_clean > 0.5).astype(int).flatten()
print(X_test.min(), X_test.max())


# only keep correctly classified traffic (TP and TN)
correct = labels_clean == Y_test[:2000]

X_correct = X_test[:2000][correct]
Y_correct = Y_test[:2000][correct]



#original predictions
preds_orig = classifier.predict(X_correct)
labels_orig = (preds_orig > 0.5).astype(int).flatten()
evaluate(Y_correct, labels_orig, "Original")

# fgsm predictions
start = time.time()
attack_fgsm = FastGradientMethod(estimator=classifier, eps=1.0)
adv_fgsm = attack_fgsm.generate(x=X_correct, y=Y_correct)
preds_adv_fgsm = classifier.predict(adv_fgsm)
labels_adv_fgsm = (preds_adv_fgsm > 0.5).astype(int).flatten()
evaluate(Y_correct, labels_adv_fgsm, "FGSM Attack")
print(f"FGSM Time: {time.time() - start:.2f} sec\n")

# ProjectedGradientDescent predictions
start = time.time()
attack_pgd = ProjectedGradientDescent(estimator=classifier, eps=1.0)
adv_pgd = attack_pgd.generate(x=X_correct, y=Y_correct)
preds_adv_pgd = classifier.predict(adv_pgd)
labels_adv_pgd = (preds_adv_pgd > 0.5).astype(int).flatten()
evaluate(Y_correct, labels_adv_fgsm, "PGD Attack")
print(f"PGD Time: {time.time() - start:.2f} sec\n")

-1.0552326 390.47467


PGD - Random Initializations: 100%|██████████| 1/1 [00:00<00:00,  7.05it/s]


--- Original ---
Accuracy : 1.0000
Precision: 1.0000
Recall   : 1.0000
F1-score : 1.0000

--- FGSM Attack ---
Accuracy : 1.0000
Precision: 1.0000
Recall   : 1.0000
F1-score : 1.0000

FGSM Time: 0.02 sec

--- FGSM Attack ---
Accuracy : 1.0000
Precision: 1.0000
Recall   : 1.0000
F1-score : 1.0000

FGSM Time: 0.01 sec

=== Prediction Counts ===
Original        : Counter({0: 1016, 1: 968})
fgsmAttack  : Counter({0: 1016, 1: 968})
ProjectedGradientDescent  : Counter({0: 1016, 1: 968})


### HopSkipJump

In [12]:
attack_hsj = HopSkipJump(classifier=classifier, norm=2,max_iter=5000, max_eval=10000, init_eval=100, init_size=100)
adv_hopskipjump = attack_hsj.generate(x=X_test[:2000])

# HopSkipJump predictions
preds_adv_hsj = classifier.predict(adv_hopskipjump)
labels_adv_hsj = (preds_adv_hsj > 0.5).astype(int).flatten()

print("=== Prediction Counts ===")
print("Original        :", Counter(labels_orig))
print("HopSkipJump     :", Counter(labels_adv_hsj))

NameError: name 'HopSkipJump' is not defined

### BoundaryAttack

In [ ]:
attack_boundary = BoundaryAttack(estimator=classifier)
adv_boundary = attack_boundary.generate(x=X_test[:2000], y=Y_test[:2000])

# BoundaryAttack predictions
preds_adv_boundary = classifier.predict(adv_boundary)
labels_adv_boundary = (preds_adv_boundary > 0.5).astype(int).flatten()

print("=== Prediction Counts ===")
print("Original        :", Counter(labels_orig))
print("BoundaryAttack  :", Counter(labels_adv_boundary))

Boundary attack: 100%|██████████| 2000/2000 [03:14<00:00, 10.27it/s]


=== Prediction Counts ===
Original        : Counter({1: 1000, 0: 1000})
BoundaryAttack  : Counter({1: 1000, 0: 1000})


### Trying Foolbox

In [ ]:
fmodel = fb.TensorFlowModel(cnn_loaded, bounds=(X_test.min(), X_test.max()))

# attack = fb.attacks.HopSkipJumpAttack()
attack = fb.attacks.BoundaryAttack()
# attack = fb.attacks.FGSM()


epsilons = [0.01, 0.05, 0.1, 0.2, 0.3, 0.6, 0.8, 1.0]
x = tf.constant(X_test[:5].astype(np.float32))
y = tf.constant(Y_test[:5].astype(np.int64))

# preds = tf.argmax(fmodel(x), axis=-1)
# mask = preds != y
# x_filtered = tf.boolean_mask(x, mask)
# y_filtered = tf.boolean_mask(y, mask)

# print(f"{len(x_filtered)} correctly classified inputs to attack")

# if len(x_filtered) == 0:
#     raise RuntimeError("No sample correctly classified to launch the attack")

advs, clipped, success = attack(fmodel, x, y, epsilons=epsilons)

for i, eps in enumerate(epsilons):
    print(f"Epsilon {eps:.2f} — Success rate: {np.mean(success[i]):.2%}")


/home/bossantm/anaconda3/envs/dl-attack-env/lib/python3.9/site-packages/foolbox/attacks/blended_noise.py:85: UserWarning: LinearSearchBlendedUniformNoiseAttack failed to draw sufficient random inputs that are adversarial (3 / 5).
  warnings.warn(


ValueError: init_attack failed for 2.0 of 5 inputs